In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import keras
from keras.layers import Activation, Input, Dense
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.datasets import mnist
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

## MNIST data prep

In [ ]:
batch_size = 64
num_classes = 10
epochs = 20

In [ ]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [ ]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=.01,decay=.000001,momentum=.9),
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

## Functional MNIST

In [ ]:
inputs = Input(shape=(784,))
output_1 = Dense(64, activation='relu')(inputs)
output_2 = Dense(64, activation='relu')(output_1)
output_3 = Dense(64, activation='relu')(output_2)
predictions = Dense(10, activation='softmax')(output_3)

In [ ]:
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
x_train = tf.reshape(x_train,[x_train.shape[0],-1])
y_train = tf.reshape(y_train,[y_train.shape[0],-1])
x_test = tf.reshape(x_test,[x_test.shape[0],-1])
y_test = tf.reshape(y_test,[y_test.shape[0],-1])

In [ ]:
model.fit(x_train, y_train,
          steps_per_epoch=batch_size,
          epochs=epochs,
          verbose=1,
          validation_steps=batch_size,
          validation_data=(x_test, y_test))

## RESNET - MNIST - 5 Heads

In [ ]:
batch_size = 64
num_classes = 10
epochs = 20

In [ ]:
# input image dimensions
inputs = Input(shape=(784,))
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
num_heads = 5

In [ ]:
def resnet_head(inputs):
    inputs = Dense(64)(inputs)
    x = BatchNormalization()(inputs)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(64)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    block_1_output = keras.layers.Add()([inputs, x])
    return block_1_output

In [ ]:
y = inputs
for heads in range(num_heads):
    y = resnet_head(y)
y = Dense(10)(y)
y = Activation('softmax')(y)

In [ ]:
# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=y)
model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
x_train = tf.reshape(x_train,[x_train.shape[0],-1])
y_train = tf.reshape(y_train,[y_train.shape[0],-1])
x_test = tf.reshape(x_test,[x_test.shape[0],-1])
y_test = tf.reshape(y_test,[y_test.shape[0],-1])

In [ ]:
model.fit(x_train, y_train,
          steps_per_epoch=batch_size,
          epochs=epochs,
          verbose=1,
          validation_steps=batch_size,
          validation_data=(x_test, y_test))

## RESNET - MNIST - 10 Heads

In [ ]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [ ]:
num_heads = 10

In [ ]:
y = inputs
for heads in range(num_heads):
    y = resnet_head(y)

In [ ]:
# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=y)
model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
x_train = tf.reshape(x_train,[x_train.shape[0],-1])
y_train = tf.reshape(y_train,[y_train.shape[0],-1])
x_test = tf.reshape(x_test,[x_test.shape[0],-1])
y_test = tf.reshape(y_test,[y_test.shape[0],-1])

In [ ]:
model.fit(x_train, y_train,
          steps_per_epoch=batch_size,
          epochs=epochs,
          verbose=1,
          validation_steps=batch_size,
          validation_data=(x_test, y_test))

## CIFAR data prep

In [ ]:
batch_size = 64
num_classes = 10
epochs = 20

In [ ]:
# input image dimensions
img_rows, img_cols = 32, 32

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 3, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 3, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 3)

In [ ]:
# This returns a tensor
inputs = Input(shape=(3072,))
#inputs = x_train
# a layer instance is callable on a tensor, and returns a tensor
#inputs = inputs.flatten()
output_1 = Dense(64, activation='relu')(inputs)
output_2 = Dense(64, activation='relu')(output_1)
output_3 = Dense(64, activation='relu')(output_2)
predictions = Dense(10, activation='softmax')(output_3)

In [ ]:
# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
x_train = tf.reshape(x_train,[x_train.shape[0],-1])
y_train = tf.reshape(y_train,[y_train.shape[0],-1])
x_test = tf.reshape(x_test,[x_test.shape[0],-1])
y_test = tf.reshape(y_test,[y_test.shape[0],-1])
print(x_train.shape)

In [ ]:
model.fit(x_train, y_train,
          steps_per_epoch=batch_size,
          epochs=epochs,
          verbose=1,
          validation_steps=batch_size,
          validation_data=(x_test, y_test))

## RESNET - CIFAR10 - 5 Heads

In [ ]:
# input image dimensions
img_rows, img_cols = 32, 32

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 3, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 3, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 3)

In [ ]:
num_heads = 5

In [ ]:
y = inputs
for heads in range(num_heads):
    y = resnet_head(y)

In [ ]:
# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
x_train = tf.reshape(x_train,[x_train.shape[0],-1])
y_train = tf.reshape(y_train,[y_train.shape[0],-1])
x_test = tf.reshape(x_test,[x_test.shape[0],-1])
y_test = tf.reshape(y_test,[y_test.shape[0],-1])
print(x_train.shape)

In [ ]:
model.fit(x_train, y_train,
          steps_per_epoch=batch_size,
          epochs=epochs,
          verbose=1,
          validation_steps=batch_size,
          validation_data=(x_test, y_test))

## RESNET - CIFAR10 - 10 Heads

In [ ]:
# input image dimensions
img_rows, img_cols = 32, 32

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 3, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 3, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 3)

In [ ]:
num_heads = 10

In [ ]:
y = inputs
for heads in range(num_heads):
    y = resnet_head(y)

In [ ]:
# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
x_train = tf.reshape(x_train,[x_train.shape[0],-1])
y_train = tf.reshape(y_train,[y_train.shape[0],-1])
x_test = tf.reshape(x_test,[x_test.shape[0],-1])
y_test = tf.reshape(y_test,[y_test.shape[0],-1])
print(x_train.shape)

In [ ]:
model.fit(x_train, y_train,
          steps_per_epoch=batch_size,
          epochs=epochs,
          verbose=1,
          validation_steps=batch_size,
          validation_data=(x_test, y_test))